In [1]:
import pandas as pd
import numpy as np


In [2]:
# для всех норм по ТПГ нужно объявить эту переменную
row_number = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,usecols=[5])

skip_row = row_number.ЦОК.tolist()
skip_row_n = skip_row.index('подгруппа')-1

In [4]:
#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ (всех)
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)
df_all = df[["статус","Номер из NAV","Адрес магазина","дата планового открытия"]]
df_all = df_all[~df_all.статус.isin(['Закрыт','закрыт'])]
df_all = df_all[['статус','Номер из NAV']].groupby(by='статус').count().reset_index(drop=True, inplace=False)

#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ иф/бк(золото)
a = [2,3,6,18] + [*range(28,41)]
a_shops = df.iloc[1:, lambda df: a]
del a_shops['ИФ аксессуары']
a_shops = a_shops[a_shops.статус.isin(['открыт','Отгружено','склад'])]
a_shops['сумма'] = a_shops.iloc[:,3:].sum(axis=1)
a_shops = a_shops[a_shops['сумма'] >0]
a_shops_count = a_shops[['статус','Номер из NAV']].groupby(by=['статус']).count().reset_index(drop=True, inplace=False)

#Переворачиваем и соединяем золото и серебро
final = df_all.rename(columns={'Номер из NAV':'с_серебром'}).T.append(a_shops_count.rename(columns={'Номер из NAV':'золото'}).T)
final.at['с_серебром',1] = final.at['с_серебром',1]-3 # удаляем 3 ломбарда
final[3]=final[0]+final[2]
final[4]=final[1]+final[3]
final = final.drop([0,2],axis=1).rename(columns={1:'открыто',3:'к_открытию',4:'итого'})

#ДОСТАЕТ СПИСОК НОВЫХ МАГОВ
new_shops = a_shops[["статус","Номер из NAV","Адрес магазина","дата планового открытия"]]
new_shops = new_shops[(new_shops.статус == 'Отгружено')|(new_shops.статус == 'склад')].drop(columns='статус').sort_values('дата планового открытия')

#ДОСТАЕТ НОРМЫ ПО магам иф/бк

norm_all=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)
list = norm_all.columns.tolist()

cols = (
    [list[list.index('Название из NAV')]] +
    list[list.index(1001):list.index(1102)+1] +
    list[list.index(1501):list.index(1597)+1]
)
opened = a_shops[['Номер из NAV','статус']]
norm_all = norm_all[cols]
merged = pd.merge(opened,norm_all,left_on='Номер из NAV', right_on='Название из NAV',how = 'left')
del merged['Название из NAV'],merged['Номер из NAV']
merged = merged.groupby(by='статус').sum()
merged = merged.T
merged = merged.assign(sum_all=merged['Отгружено']+merged['открыт'] + merged['склад'])
del merged['Отгружено'],merged['склад']

# СОХРАНЯЕМ ВСЕ В 1 ФАЙЛ

with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\для_новые_нормы.xlsx'
) as writer:
    final.to_excel(writer, sheet_name='количество')
    new_shops.to_excel(writer, sheet_name='новые_маги', index=False)
    merged.to_excel(writer, sheet_name='новые_нормы')


In [4]:
#ДОСТАЕТ НОРМЫ ПО БУКВАМ И ЗОДИАКАМ 

# Читаем реестр магов
norm = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12', skiprows=4, nrows=skip_row_n)

norm.set_index('Название из NAV',inplace=True)

b_columns = [1088, 1089, 1090, 1586, 1587, 1588]
z_columns = [1091, 1092, 1093, 1589, 1590, 1591]

norm_b = norm[b_columns]
norm_z = norm[z_columns]

norm_b = norm_b.assign(sum_if = norm_b[1088] + norm_b[1089] + norm_b[1090],
              sum_bk = norm_b[1586] + norm_b[1587] + norm_b[1588])

norm_z = norm_z.assign(sum_if1 = norm_z[1091] + norm_z[1092] + norm_z[1093],
              sum_bk1 = norm_z[1589] + norm_z[1590] + norm_z[1591])

norm_z = norm_z[norm_z.columns[[0,1,2,6,3,4,5,7]]]

with pd.ExcelWriter (r'C:\Остатки\norm_bz.xlsx') as writer:
    norm_b.to_excel(writer, sheet_name='norm_b')
    norm_z.to_excel(writer, sheet_name='norm_z')

In [5]:
#ВЫДАЕТ СПИСОК АКТУАЛЬНЫХ МАГОВ ИФ\БК С НОРМАМИ ПО ТГ И ДАТЫ ОТКРЫТИЯ

#выгружаем датафрэйм с актуальными нормами ИФ БК из реестра магов 
all_shops = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=[2,3,18]+[*range(28,34)]+[*range(35,41)])

#выбираем открытые маги с суммой норм по иф/бк больше нуля
all_shops = all_shops[all_shops.статус.isin(['открыт','Отгружено','склад'])]
all_shops['сумма'] = all_shops.iloc[:,3:].sum(axis=1)
all_shops = all_shops[all_shops['сумма'] >0]
del all_shops['статус'], all_shops['сумма']
all_shops = all_shops.rename(columns={'Номер из NAV':'Код'})
all_shops['Код'] = all_shops['Код'].astype(np.int64)

# делим на два отдельных иф и бк
cols = all_shops.columns.tolist()
shops_if = all_shops[cols[:5]+[cols[7],cols[6],cols[5]]]
shops_bk = all_shops[cols[:2]+cols[8:11]+[cols[13],cols[12],cols[11]]]

#выгружаем из файла "наполняемость" актуальную структуру магов иф/бк, добавляет туда нормы и новые маги из датафрэйма выше
shops_if_list = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='ИФ',skiprows=3, usecols=[2],keep_default_na=False)

shops_bk_list = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='БК',skiprows=3, usecols=[2],keep_default_na=False)

# список новых магов которые надо добавить
add_to_if = pd.DataFrame({'Код':[x for x in shops_if['Код'].tolist() if x not in shops_if_list['Код'].tolist()]})
add_to_bk = pd.DataFrame({'Код':[x for x in shops_bk['Код'].tolist() if x not in shops_bk_list['Код'].tolist()]})

# добавляем новые маги в структуру
shops_if_list = shops_if_list.append(add_to_if, sort=False)
shops_bk_list = shops_bk_list.append(add_to_bk, sort=False)

#соединяем "структуру" и нормы
shops_if = pd.merge(shops_if_list,shops_if,how='left',on='Код')
shops_bk = pd.merge(shops_bk_list,shops_bk,how='left',on='Код')


#удаляем индекс и сохраняем на отдельные вкладки
shops_if.set_index('Код', drop=True,inplace=True)
shops_bk.set_index('Код', drop=True,inplace=True)

#создаем датафрейм из магов с датами открытия последние 90 дней
from datetime import date, timedelta

three_month_ago = date.today() - timedelta(90)
dates=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols = [2,6,18])
dates.rename(columns={'дата планового открытия':'дата'}, inplace=True)

dates=dates[(
    dates.статус == 'Отгружено')|(
    dates.статус == 'склад')|(
    dates.статус == 'открыт')&(
    dates.дата > pd.Timestamp(three_month_ago
                             ))]
del dates['статус']
dates['дата'] = dates.дата.apply(lambda x:x.date())
dates.set_index('Номер из NAV', drop=True,inplace=True)

#сохраняем все в эксель
with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\актуальные маги.xlsx'
) as writer:
    shops_if.to_excel(writer, sheet_name='ИФ')
    shops_bk.to_excel(writer, sheet_name='БК')
    dates.to_excel(writer, sheet_name='даты')

In [13]:
# СПИСОК НОРМ ПО ТГ

#читает файл с реестром
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)

#вынимает нужные две строки и сохраняет
rep_tn = df.iloc[0:1,28:73]
rep_tn.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\Рейтинг ТГ и ТН\123.xlsx',
index=False)

In [30]:
np.random.seed(32)
df1 = pd.DataFrame({'1516':np.random.randint(1,3,size=10), '1517':np.random.randint(2,4,size=10)})
df2 = pd.DataFrame({'1516':np.random.randint(2,size=10), '1517':np.random.randint(3,size=10)})
df3 = df1 - df2 
print(df1)
print(df2)
print(df3)

   1516  1517
0     2     2
1     2     3
2     2     3
3     1     3
4     1     2
5     1     3
6     1     2
7     2     3
8     1     3
9     2     3
   1516  1517
0     1     0
1     0     1
2     1     1
3     0     1
4     0     2
5     0     1
6     0     2
7     0     0
8     0     2
9     0     1
   1516  1517
0     1     2
1     2     2
2     1     2
3     1     2
4     1     0
5     1     2
6     1     0
7     2     3
8     1     1
9     2     2
